In [1]:
import numpy as np
import nibabel as nib
import random

In [2]:
def generate_3d_volume(image_size, radius, center_sphere):
    # Create a 3D grid
    x = np.linspace(0, 1, image_size[0])
    y = np.linspace(0, 1, image_size[1])
    z = np.linspace(0, 1, image_size[2])
    x, y, z = np.meshgrid(x, y, z, indexing='ij')  # Ensure correct indexing for the grid

    # Sphere mask using the fixed radius and center position
    sphere_mask = ((x - center_sphere[0])**2 + (y - center_sphere[1])**2 + (z - center_sphere[2])**2) < (radius / image_size[0])**2


    # Initialize volume with grey for background
    volume = np.full(image_size, 255)  # Set entire volume to grey

    # Set the sphere to white
    volume[sphere_mask] = 128  # Make the sphere white

    return volume

In [3]:
def gen_samples(batch_size, radius, image_size):
    while True:
        vols = []
        for _ in range(batch_size):
            sphere_center = (
                random.uniform(radius / image_size[0], 1 - radius / image_size[0]),
                random.uniform(radius / image_size[1], 1 - radius / image_size[1]),
                random.uniform(radius / image_size[2], 1 - radius / image_size[2])
            )
            vol = generate_3d_volume(image_size, radius, sphere_center)
            vol = np.expand_dims(vol, axis=-1)  # Add channel dimension
            vols.append(vol)
        yield np.array(vols),np.array(vols)

In [4]:
def define_autoencoder(input_shape):

    input_layer = Input(shape=input_shape)
    
    
    # Encoder
    encoder = Conv3D(64, kernel_size=(3, 3, 3), activation='relu', padding='same')(input_layer)
    # exit(1)
    
    encoder = BatchNormalization()(encoder)
    encoder = MaxPooling3D(pool_size=(2, 2, 2), padding='same')(encoder)
    
    
    
    encoder = Conv3D(128, kernel_size=(3, 3, 3), activation='relu', padding='same')(encoder)
    
    encoder = BatchNormalization()(encoder)
    encoder = MaxPooling3D(pool_size=(2, 2, 2), padding='same')(encoder)
    # print('####encoder###',encoder)
  
    
    # Decoder
    decoder = Conv3DTranspose(128, kernel_size=(3, 3, 3), activation='relu', padding='same')(encoder)
    decoder = BatchNormalization()(decoder)
    decoder = tf.keras.layers.UpSampling3D(size=(2, 2, 2))(decoder)
    
    
    
    decoder = Conv3DTranspose(64, kernel_size=(3, 3, 3), activation='relu', padding='same')(decoder)
    decoder = BatchNormalization()(decoder)
    decoder = tf.keras.layers.UpSampling3D(size=(2, 2, 2))(decoder)
    
   
    
    
    # Output layer with the same number of channels as input data
    output_layer = Conv3D(1, kernel_size=(3, 3, 3), activation='sigmoid', padding='same')(decoder)
    output_layer = output_layer * 255
   
    
    
    
    autoencoder = Model(inputs=input_layer, outputs=output_layer)
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    autoencoder.compile(optimizer= optimizer, loss= tf.keras.losses.MeanSquaredError())  # Adjust loss function if needed
    

    
    return autoencoder


In [6]:
import os
import tensorflow as tf
import shutil
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, Conv3DTranspose, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard, Callback
import h5py
from datetime import datetime
import numpy as np
import pdb
import matplotlib.pyplot as plt
import random
from tqdm.keras import TqdmCallback



# # Load your npz file
# loaded_data = np.load('synthetic_dataset.npz')
# num_samples = len(loaded_data.files)
# sample_fraction = 1
# num_samples_to_use = int(num_samples * sample_fraction)


# # Use the loaded_data dictionary to get the volumes
# train_x = np.array([loaded_data[f'arr_{i}'] for i in range(num_samples_to_use)])
# print(list(set([np.max(train_x[i]) for i in range(len(train_x))])))
# print(train_x.shape)
# # exit(1)

# # Reshape train_x to add a channel dimension, assuming grayscale images so only 1 channel is added
# train_x = np.expand_dims(train_x, axis=-1)
# print("New train_x shape with channel dimension:", train_x.shape)

# # Define autoencoder architecture




# # Instantiate and compile autoencoder
# input_shape = train_x.shape[1:]
# print("the last input shape", train_x.shape[1:])

# autoencoder = define_autoencoder(input_shape)

# # # Create a tf.data.Dataset from the numpy array
# # dataset = tf.data.Dataset.from_tensor_slices(train_x)  # Changed line
# # batch_size = 10  # Added line
# # dataset = dataset.shuffle(buffer_size=1000).batch(batch_size)  # Changed line


image_size = (16, 16, 16)  
radius = 4 
batch_size = 10
input_shape = (16,16,16,1)
autoencoder = define_autoencoder(input_shape)


dataset = tf.data.Dataset.from_generator(
    lambda: gen_samples(batch_size, radius, image_size),
    output_types=(tf.float32, tf.float32),  
    output_shapes=([None, *input_shape], [None, *input_shape])
)

# Configure the dataset for performance
# dataset = dataset.batch(batch_size)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)



steps_per_epoch = 1000 // batch_size
# Train the autoencoder using the dataset
autoencoder.fit(dataset, steps_per_epoch = steps_per_epoch, epochs=100)  # Changed line


autoencoder.save('autoencoder_model.h5')







Epoch 1/100
100/100 [==============================] - 3s 22ms/step - loss: 2728.7786
Epoch 2/100
100/100 [==============================] - 2s 22ms/step - loss: 815.5796
Epoch 3/100
100/100 [==============================] - 2s 21ms/step - loss: 942.7221
Epoch 4/100
100/100 [==============================] - 2s 22ms/step - loss: 785.2240
Epoch 5/100
100/100 [==============================] - 2s 21ms/step - loss: 631.8572
Epoch 6/100
100/100 [==============================] - 2s 21ms/step - loss: 377.3864
Epoch 7/100
100/100 [==============================] - 2s 22ms/step - loss: 251.6550
Epoch 8/100
100/100 [==============================] - 2s 21ms/step - loss: 170.8270
Epoch 9/100
100/100 [==============================] - 2s 22ms/step - loss: 119.1774
Epoch 10/100
100/100 [==============================] - 2s 21ms/step - loss: 101.5331
Epoch 11/100
100/100 [==============================] - 2s 21ms/step - loss: 88.6907
Epoch 12/100
100/100 [==============================] - 2s 22ms

In [8]:
# Assuming dataset is your tf.data.Dataset object from earlier
for inputs, _ in dataset.take(1):  # Take one batch from the dataset
    sample_volume = inputs.numpy()[0]  # Convert the first sample of the batch to a NumPy array

    # Ensure the volume is correctly shaped for NIfTI saving
    # Your volume might already be in the correct shape, but verify this.
    # If your data has an extra channel dimension at the end (e.g., shape is (X, Y, Z, 1) for grayscale),
    # you might need to squeeze it to (X, Y, Z) if that's unnecessary.
    sample_volume = np.squeeze(sample_volume)

    # Create a NIfTI image. Assuming no specific affine transformation, use an identity matrix.
    # Adjust the affine matrix if you have specific spatial orientation requirements.
    nifti_img = nib.Nifti1Image(sample_volume, affine=np.eye(4))

    # Save the NIfTI image to disk
    nib.save(nifti_img, 'sample_volume.nii.gz')

In [9]:
import nibabel as nib
import numpy as np

# Load the NIfTI file
nifti_sample_path = 'sample_volume.nii.gz'
nifti_sample = nib.load(nifti_sample_path)

# Get the data as a numpy array
sample_volume = nifti_sample.get_fdata()

# Depending on your model's expected input shape, you might need to add a batch dimension
# and possibly a channel dimension if your model expects it. Here's an example:
sample_volume = np.expand_dims(sample_volume, axis=0)  # Add batch dimension
sample_volume = np.expand_dims(sample_volume, axis=-1)  # Add channel dimension if needed

# Ensure the sample data type matches your model's expected input type, typically float32
sample_volume = sample_volume.astype(np.float32)


# Assuming autoencoder is your trained model
predicted_output = autoencoder.predict(sample_volume)




# Remove batch dimension and channel dimension if added before
predicted_output = np.squeeze(predicted_output)

# Create a NIfTI image for the output
# Use the same affine transformation as the input for consistency
output_nifti_img = nib.Nifti1Image(predicted_output, affine=nifti_sample.affine)

# Save the output NIfTI image
nib.save(output_nifti_img, 'predicted_output.nii.gz')
